In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import sys
import random
import os
import re
import math
import numpy as np
import pandas as pd
import xlwt

import cv2
# import pyautogui
import imutils
import urllib
import datetime

In [2]:
# 입력
cnt = 1000
real_cnt = math.ceil(cnt / 100)

# 번개 들어가기
path = 'C:/temp/chromedriver.exe'
driver = webdriver.Chrome(path)

url = "https://m.bunjang.co.kr/"
driver.get(url)
time.sleep(2)
driver.maximize_window()

In [3]:
# 자동 스크롤다운 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6500);")
    time.sleep(5)

def scroll_up(driver):
    driver.execute_script("window.scrollTo(0,-6500);")
    time.sleep(5)


def scroll_down_oC(driver) :
    match_category_1 = match_center_loc(screenshot_1, img_down)
    pyautogui.moveTo(match_category_1)
    pyautogui.click()
    # time.sleep(2)

def match_center_loc(screen, template):
    res = cv2.matchTemplate(screen, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    top_left = max_loc
    h, w = template.shape[:2]
    x, y = int(top_left[0] + w/2), int(top_left[1] + h/2)
    return x,y

def url_pa(max_page_num):
    
    all_page_num = 1    
    click_num = 1
    page_num = 2
    

    img_link = [] # 추가 코드
    while(click_num <= real_cnt):
        scroll_down(driver)
        # doScrollDown_second(30)
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # dress_all = soup.find('div', 'sc-cKZAiZ murgl').find_all('a')
        dress_all = soup.find('div', 'sc-bOCYYb jZFdFX').find_all('a')
        

        # img_link = [] #기존 코드
        url_count = 1
        for dress_img in dress_all:
            try:
                img_link.append(dress_img['href'])
            except:
                continue
            else:
                print(f'{url_count} 번째 url 입니다.')
                url_count += 1
            if url_count == cnt :
                break

        if all_page_num == max_page_num:
            break
        
        if page_num == 11 : 
            driver.find_element_by_xpath('//*[@id="root"]/div/div/div[5]/div/div[5]/div/a[12]').click()
            page_num = 3
        elif page_num % 10 != 0:
            driver.find_element_by_xpath(f'//*[@id="root"]/div/div/div[5]/div/div[5]/div/a[{page_num}]').click()
   
        page_num += 1
        all_page_num += 1

        

        time.sleep(5)
    return img_link # 추가 코드

def doScrollDown_second(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        driver.execute_script('window.scrollTo(0, 1000);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

In [6]:
import time
import pickle
start_time = time.time()  # 시작 시간 저장

# 여성의류까지 들어가기
driver.find_element_by_xpath('//*[@id="root"]/div/div/div[3]/div[1]/div[2]/div/img').click()
driver.find_element_by_link_text('여성의류').click()
time.sleep(1)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
b_list = soup.find('div', 'sc-hTQSVH lhjSZQ').find_all('div','sc-ldcLGC hShMnB')

#     driver.find_element_by_xpath('//*[@id="root"]/div/div/div[5]/div/div[2]/div/div[1]/a').click() #제일 앞에 소분류 클릭

#     time.sleep(2)

full_url = []
url_link = url_pa(20) # url 가져오기



for url_piece in url_link:
    url = 'https://m.bunjang.co.kr' + url_piece
    full_url.append(url)

# 오류 대비 URL 저장
with open('temp.pic', 'wb') as f:
    pickle.dump(full_url, f, pickle.HIGHEST_PROTOCOL)
    
# 리스트 정의
title_list = []
category_big_list = []
category_mid_list = []
price_list = []
detail_list = []
region_list = []
file_no_list = []
status_list = []
url_all_list = []
img_id_list = []

# 경로 정의 및 생성
f_dir = "D:/bunjang/data"

# 각 페이지 들어가기
for c in range(0, len(full_url)) :
    
    category_all = []
    # 카테고리가 없어지는 버그가 일어날 시 페이지 새로고침
    try_count = 0
    while len(category_all) == 0:
        driver.get(full_url[c])
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        category_all = soup.select('div.sc-fxgLge.hUWMqp')
        # 무한루프 탈출
        try_count += 1
        if try_count > 4:
            driver.back()
            break
    try:
        # 제목, 가격, 상품정보, 장소, 상품상태
        title = soup.select_one('div.sc-kyCyAI.gXBiJn').get_text()
        price = soup.select_one('div.sc-fEVUGC.eCNjoQ > div').get_text().replace(" ","").replace("\n", "")
        detail = soup.select_one('div.sc-kqEXUp.dIqkOL').get_text().replace("\n", "")
        # 장소인증/인증 안된거 예외 처리
        try:
            region = soup.select_one('div.sc-hQfrgq.iwpbSD > span').get_text()
        except:
            region = soup.select_one('div.sc-hQfrgq.iwpbSD > div').get_text()
        status = soup.select_one('div.sc-csSMhA.eNLlHF').get_text()

        # 카테고리 세분화
        category = []
        for cat in category_all:
            category.append(cat.get_text().replace(" ", "").replace("/","_"))

        os.makedirs(f_dir + "/" + category[0] + '/' + str(category[1]) , exist_ok=True)

        # 이미지 저장하는 부분
        img_link = []
        img_all = soup.select('div.sc-lhLRcH.ggOZCK > img')
        for img in img_all:
            img_link.append(img['src'])

        file_no = 1

        img_id = full_url[c].split('/')[4].split('?')[0]

        for link in img_link:
            urllib.request.urlretrieve(link, f_dir + '/' + category[0] + '/' + category[1] + '/' + img_id + '_' + str(file_no) + '.jpg')
            print(f'{c+1}번째 게시글 {file_no} 번째 이미지 저장 완료')
            file_no += 1
        file_no_list.append(file_no-1)

        #제목, 가격, 카테고리, 설명 리스트에 넣기
        img_id_list.append(img_id)
        title_list.append(title)
        category_big_list.append(category[0])
        category_mid_list.append(category[1])
        price_list.append(price)
        detail_list.append(detail)
        region_list.append(region)
        status_list.append(status)
        url_all_list.append(full_url[c])
        category_list.append(category)

        print(f'{c+1} 번째 게시글 정보 저장 완료')
    except: 
        driver.back()
        continue


# pandas dataframe 정의
data = {'img_id': img_id_list,'title': title_list, 'category_big': category_big_list, 'category_mid': category_mid_list, 'price':price_list, 'detail':detail_list, 'region':region_list, 'status':status_list, 'file_count':file_no_list, 'url':url_all_list}
df = pd.DataFrame(data)


# csv로 저장
df.to_csv(f_dir + '/' + category_big_list[0] + '/' + category_mid_list[0] + '/' + category_mid_list[0] + '_meta_data.csv')

#한바퀴 돌리는데 들은 시간 표시
print("time :", time.time() - start_time)
    
driver.get("https://m.bunjang.co.kr/categories/310?req_ref=popular_category")
    



번째 이미지 저장 완료
1821번째 게시글 3 번째 이미지 저장 완료
1821번째 게시글 4 번째 이미지 저장 완료
1822번째 게시글 1 번째 이미지 저장 완료
1822번째 게시글 2 번째 이미지 저장 완료
1822번째 게시글 3 번째 이미지 저장 완료
1823번째 게시글 1 번째 이미지 저장 완료
1823번째 게시글 2 번째 이미지 저장 완료
1824번째 게시글 1 번째 이미지 저장 완료
1825번째 게시글 1 번째 이미지 저장 완료
1825번째 게시글 2 번째 이미지 저장 완료
1826번째 게시글 1 번째 이미지 저장 완료
1826번째 게시글 2 번째 이미지 저장 완료
1826번째 게시글 3 번째 이미지 저장 완료
1827번째 게시글 1 번째 이미지 저장 완료
1827번째 게시글 2 번째 이미지 저장 완료
1828번째 게시글 1 번째 이미지 저장 완료
1828번째 게시글 2 번째 이미지 저장 완료
1828번째 게시글 3 번째 이미지 저장 완료
1828번째 게시글 4 번째 이미지 저장 완료
1829번째 게시글 1 번째 이미지 저장 완료
1829번째 게시글 2 번째 이미지 저장 완료
1830번째 게시글 1 번째 이미지 저장 완료
1831번째 게시글 1 번째 이미지 저장 완료
1831번째 게시글 2 번째 이미지 저장 완료
1831번째 게시글 3 번째 이미지 저장 완료
1831번째 게시글 4 번째 이미지 저장 완료
1832번째 게시글 1 번째 이미지 저장 완료
1832번째 게시글 2 번째 이미지 저장 완료
1832번째 게시글 3 번째 이미지 저장 완료
1832번째 게시글 4 번째 이미지 저장 완료
1833번째 게시글 1 번째 이미지 저장 완료
1833번째 게시글 2 번째 이미지 저장 완료
1833번째 게시글 3 번째 이미지 저장 완료
1833번째 게시글 4 번째 이미지 저장 완료
1833번째 게시글 5 번째 이미지 저장 완료
1834번째 게시글 1 번째 이미지 저장 완료
1834번째 게시글 2 번째 이미지 저장 완료
1835번째 게시글 1 번째 이미지 저장 완료

In [ ]:
import time
import pickle
start_time = time.time()  # 시작 시간 저장

# 여성의류까지 들어가기
driver.find_element_by_xpath('//*[@id="root"]/div/div/div[3]/div[1]/div[2]/div/img').click()
driver.find_element_by_link_text('여성의류').click()
time.sleep(1)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
b_list = soup.find('div', 'sc-hTQSVH lhjSZQ').find_all('div','sc-ldcLGC hShMnB')

# !! 숫자 정하기 !! 1 ~ 5, 6 ~ 10, 11 ~ 15, 16 ~ len(b_list)
start_num = 10
end_num = 10


for b_num in range(start_num, end_num + 1): #len(b_list)
    driver.find_element_by_xpath('//*[@id="root"]/div/div/div[5]/div/div[2]/div/div[%s]/a/div[1]' % b_num).click() #중분류 클릭
    time.sleep(3)

    #     driver.find_element_by_xpath('//*[@id="root"]/div/div/div[5]/div/div[2]/div/div[1]/a').click() #제일 앞에 소분류 클릭

    #     time.sleep(2)

    full_url = []
    url_link = url_pa(20) # url 가져오기
    
   

    for url_piece in url_link:
        url = 'https://m.bunjang.co.kr' + url_piece
        full_url.append(url)

    # 오류 대비 URL 저장
    with open('temp.pic', 'wb') as f:
        pickle.dump(full_url, f, pickle.HIGHEST_PROTOCOL)
        
    # 리스트 정의
    title_list = []
    category_big_list = []
    category_mid_list = []
    price_list = []
    detail_list = []
    region_list = []
    file_no_list = []
    status_list = []
    url_all_list = []
    img_id_list = []

    # 경로 정의 및 생성
    f_dir = "D:/bunjang/data"

    # 각 페이지 들어가기
    for c in range(0, len(full_url)) :
        
        category_all = []
        # 카테고리가 없어지는 버그가 일어날 시 페이지 새로고침
        try_count = 0
        while len(category_all) == 0:
            driver.get(full_url[c])
            time.sleep(2)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            category_all = soup.select('div.sc-fxgLge.hUWMqp')
            # 무한루프 탈출
            try_count += 1
            if try_count > 4:
                driver.back()
                break
        try:
            # 제목, 가격, 상품정보, 장소, 상품상태
            title = soup.select_one('div.sc-kyCyAI.gXBiJn').get_text()
            price = soup.select_one('div.sc-fEVUGC.eCNjoQ > div').get_text().replace(" ","").replace("\n", "")
            detail = soup.select_one('div.sc-kqEXUp.dIqkOL').get_text().replace("\n", "")
            # 장소인증/인증 안된거 예외 처리
            try:
                region = soup.select_one('div.sc-hQfrgq.iwpbSD > span').get_text()
            except:
                region = soup.select_one('div.sc-hQfrgq.iwpbSD > div').get_text()
            status = soup.select_one('div.sc-csSMhA.eNLlHF').get_text()

            # 카테고리 세분화
            category = []
            for cat in category_all:
                category.append(cat.get_text().replace(" ", "").replace("/","_"))

            os.makedirs(f_dir + "/" + category[0] + '/' + str(category[1]) , exist_ok=True)

            # 이미지 저장하는 부분
            img_link = []
            img_all = soup.select('div.sc-lhLRcH.ggOZCK > img')
            for img in img_all:
                img_link.append(img['src'])

            file_no = 1

            img_id = full_url[c].split('/')[4].split('?')[0]

            for link in img_link:
                urllib.request.urlretrieve(link, f_dir + '/' + category[0] + '/' + category[1] + '/' + img_id + '_' + str(file_no) + '.jpg')
                print(f'{c+1}번째 게시글 {file_no} 번째 이미지 저장 완료')
                file_no += 1
            file_no_list.append(file_no-1)

            제목, 가격, 카테고리, 설명 리스트에 넣기
            img_id_list.append(img_id)
            title_list.append(title)
            category_big_list.append(category[0])
            category_mid_list.append(category[1])
            price_list.append(price)
            detail_list.append(detail)
            region_list.append(region)
            status_list.append(status)
            url_all_list.append(full_url[c])
            category_list.append(category)

            print(f'{c+1} 번째 게시글 정보 저장 완료')
        except: 
            driver.back()
            continue


    # pandas dataframe 정의
    data = {'img_id': img_id_list,'title': title_list, 'category_big': category_big_list, 'category_mid': category_mid_list, 'price':price_list, 'detail':detail_list, 'region':region_list, 'status':status_list, 'file_count':file_no_list, 'url':url_all_list}
    df = pd.DataFrame(data)
    

    # csv로 저장
    df.to_csv(f_dir + '/' + category_big_list[0] + '/' + category_mid_list[0] + '/' + category_mid_list[0] + '_meta_data.csv')

    #한바퀴 돌리는데 들은 시간 표시
    print("time :", time.time() - start_time)
        
    driver.get("https://m.bunjang.co.kr/categories/310?req_ref=popular_category")
        



In [5]:
# 오류시 URL 불러오기
# 오류 대비 URL 저장
import pickle
with open('temp.pic', 'rb') as f:
    full_url = pickle.load(f)

# 리스트 정의
title_list = []
category_big_list = []
category_mid_list = []
price_list = []
detail_list = []
region_list = []
file_no_list = []
status_list = []
url_all_list = []
img_id_list = []

# 경로 정의 및 생성
f_dir = "D:/bunjang/data"

# 각 페이지 들어가기
for c in range(1211, len(full_url)) :

    category_all = []
    try_count = 0
    # 카테고리가 없어지는 버그가 일어날 시 페이지 새로고침
    while len(category_all) == 0:
        driver.get(full_url[c])
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        category_all = soup.select('div.sc-fxgLge.hUWMqp')
         # 무한루프 탈출
        try_count += 1
        if try_count > 4:
            driver.back()
            break
    try:
        # 제목, 가격, 상품정보, 장소, 상품상태
        title = soup.select_one('div.sc-kyCyAI.gXBiJn').get_text()
        price = soup.select_one('div.sc-fEVUGC.eCNjoQ > div').get_text().replace(" ","").replace("\n", "")
        detail = soup.select_one('div.sc-kqEXUp.dIqkOL').get_text().replace("\n", "")
        # 장소인증/인증 안된거 예외 처리
        try:
            region = soup.select_one('div.sc-hQfrgq.iwpbSD > span').get_text()
        except:
            region = soup.select_one('div.sc-hQfrgq.iwpbSD > div').get_text()
        status = soup.select_one('div.sc-csSMhA.eNLlHF').get_text()

        # 카테고리 세분화
        category = []
        for cat in category_all:
            category.append(cat.get_text().replace(" ", "").replace("/","_"))

        os.makedirs(f_dir + "/" + category[0] + '/' + str(category[1]) , exist_ok=True)

        # 이미지 저장하는 부분
        img_link = []
        img_all = soup.select('div.sc-lhLRcH.ggOZCK > img')
        for img in img_all:
            img_link.append(img['src'])

        file_no = 1

        img_id = full_url[c].split('/')[4].split('?')[0]

        for link in img_link:
            urllib.request.urlretrieve(link, f_dir + '/' + category[0] + '/' + category[1] + '/' + img_id + '_' + str(file_no) + '.jpg')
            print(f'{c+1}번째 게시글 {file_no} 번째 이미지 저장 완료')
            file_no += 1
        file_no_list.append(file_no-1)

        # 제목, 가격, 카테고리, 설명 리스트에 넣기
        img_id_list.append(img_id)
        title_list.append(title)
        category_big_list.append(category[0])
        category_mid_list.append(category[1])
        price_list.append(price)
        detail_list.append(detail)
        region_list.append(region)
        status_list.append(status)
        url_all_list.append(full_url[c])
        # category_list.append(category)


        print(f'{c+1} 번째 게시글 정보 저장 완료')
    except: 
        driver.back()
        continue


# pandas dataframe 정의
data = {'img_id': img_id_list,'title': title_list, 'category_big': category_big_list, 'category_mid': category_mid_list, 'price':price_list, 'detail':detail_list, 'region':region_list, 'status':status_list, 'file_count':file_no_list, 'url':url_all_list}
df = pd.DataFrame(data)

# csv로 저장
df.to_csv(f_dir + '/' + category_big_list[0] + '/' + category_mid_list[0] + '/' + category_mid_list[0] + '_meta_data_4.csv')

#한바퀴 돌리는데 들은 시간 표시
print("time :", time.time() - start_time)

driver.get("https://m.bunjang.co.kr/categories/310?req_ref=popular_category")

이미지 저장 완료
1830번째 게시글 3 번째 이미지 저장 완료
1830번째 게시글 4 번째 이미지 저장 완료
1830번째 게시글 5 번째 이미지 저장 완료
1830번째 게시글 6 번째 이미지 저장 완료
1830번째 게시글 7 번째 이미지 저장 완료
1830번째 게시글 8 번째 이미지 저장 완료
1830번째 게시글 9 번째 이미지 저장 완료
1830 번째 게시글 정보 저장 완료
1831번째 게시글 1 번째 이미지 저장 완료
1831번째 게시글 2 번째 이미지 저장 완료
1831 번째 게시글 정보 저장 완료
1832번째 게시글 1 번째 이미지 저장 완료
1832 번째 게시글 정보 저장 완료
1833번째 게시글 1 번째 이미지 저장 완료
1833번째 게시글 2 번째 이미지 저장 완료
1833번째 게시글 3 번째 이미지 저장 완료
1833 번째 게시글 정보 저장 완료
1834번째 게시글 1 번째 이미지 저장 완료
1834 번째 게시글 정보 저장 완료
1835번째 게시글 1 번째 이미지 저장 완료
1835번째 게시글 2 번째 이미지 저장 완료
1835 번째 게시글 정보 저장 완료
1836번째 게시글 1 번째 이미지 저장 완료
1836번째 게시글 2 번째 이미지 저장 완료
1836 번째 게시글 정보 저장 완료
1837번째 게시글 1 번째 이미지 저장 완료
1837 번째 게시글 정보 저장 완료
1838번째 게시글 1 번째 이미지 저장 완료
1838번째 게시글 2 번째 이미지 저장 완료
1838 번째 게시글 정보 저장 완료
1839번째 게시글 1 번째 이미지 저장 완료
1839번째 게시글 2 번째 이미지 저장 완료
1839번째 게시글 3 번째 이미지 저장 완료
1839 번째 게시글 정보 저장 완료
1840번째 게시글 1 번째 이미지 저장 완료
1840번째 게시글 2 번째 이미지 저장 완료
1840번째 게시글 3 번째 이미지 저장 완료
1840 번째 게시글 정보 저장 완료
1841번째 게시글 1 번째 이미지 저장 완료
1841번째 게시글 2 번째 이미지 저장 완료
1841번

NameError: name 'start_time' is not defined

In [11]:
# pandas dataframe 정의
data = {'img_id': img_id_list,'title': title_list, 'category_big': category_big_list, 'category_mid': category_mid_list, 'price':price_list, 'detail':detail_list, 'region':region_list, 'status':status_list, 'file_count':file_no_list, 'url':url_all_list}
df = pd.DataFrame(data)

# csv로 저장
df.to_csv(f_dir + '/' + category_big_list[0] + '/' + category_mid_list[0] + '/' + category_mid_list[0] + '_meta_data4.csv')

In [ ]:
# 오류시 URL 불러오기
# 오류 대비 URL 저장
import pickle
with open('temp.pic', 'rb') as f:
    full_url = pickle.load(f)

# 리스트 정의
title_list = []
category_big_list = []
category_mid_list = []
price_list = []
detail_list = []
region_list = []
file_no_list = []
status_list = []
url_all_list = []
img_id_list = []

# 경로 정의 및 생성
f_dir = "D:/bunjang/data"

# 각 페이지 들어가기
for c in range(1109, len(full_url)) :

    category_all = []
    # 카테고리가 없어지는 버그가 일어날 시 페이지 새로고침
    while len(category_all) == 0:
        driver.get(full_url[c])
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        category_all = soup.select('div.sc-fxgLge.hUWMqp')
    try:
        # 제목, 가격, 상품정보, 장소, 상품상태
        title = soup.select_one('div.sc-kyCyAI.gXBiJn').get_text()
        price = soup.select_one('div.sc-fEVUGC.eCNjoQ > div').get_text().replace(" ","").replace("\n", "")
        detail = soup.select_one('div.sc-kqEXUp.dIqkOL').get_text().replace("\n", "")
        # 장소인증/인증 안된거 예외 처리
        try:
            region = soup.select_one('div.sc-hQfrgq.iwpbSD > span').get_text()
        except:
            region = soup.select_one('div.sc-hQfrgq.iwpbSD > div').get_text()
        status = soup.select_one('div.sc-csSMhA.eNLlHF').get_text()

        # 카테고리 세분화
        category = []
        for cat in category_all:
            category.append(cat.get_text().replace(" ", "").replace("/","_"))

        os.makedirs(f_dir + "/" + category[0] + '/' + str(category[1]) , exist_ok=True)

        # 이미지 저장하는 부분
        img_link = []
        img_all = soup.select('div.sc-lhLRcH.ggOZCK > img')
        for img in img_all:
            img_link.append(img['src'])

        file_no = 1

        img_id = full_url[c].split('/')[4].split('?')[0]

        for link in img_link:
            urllib.request.urlretrieve(link, f_dir + '/' + category[0] + '/' + category[1] + '/' + img_id + '_' + str(file_no) + '.jpg')
            print(f'{c+1}번째 게시글 {file_no} 번째 이미지 저장 완료')
            file_no += 1
        file_no_list.append(file_no-1)

        # 제목, 가격, 카테고리, 설명 리스트에 넣기
        img_id_list.append(img_id)
        title_list.append(title)
        category_big_list.append(category[0])
        category_mid_list.append(category[1])
        price_list.append(price)
        detail_list.append(detail)
        region_list.append(region)
        status_list.append(status)
        url_all_list.append(full_url[c])
        # category_list.append(category)


        print(f'{c+1} 번째 게시글 정보 저장 완료')
    except: 
        driver.back()
        continue


# pandas dataframe 정의
data = {'img_id': img_id_list,'title': title_list, 'category_big': category_big_list, 'category_mid': category_mid_list, 'price':price_list, 'detail':detail_list, 'region':region_list, 'status':status_list, 'file_count':file_no_list, 'url':url_all_list}
df = pd.DataFrame(data)

# csv로 저장
df.to_csv(f_dir + '/' + category_big_list[0] + '/' + category_mid_list[0] + '/' + category_mid_list[0] + '_meta_data_2.csv')

#한바퀴 돌리는데 들은 시간 표시
print("time :", time.time() - start_time)

driver.get("https://m.bunjang.co.kr/categories/310?req_ref=popular_category")

In [3]:
# pandas dataframe 정의
import pandas as pd
data = {'img_id': ,'title': , 'category_big': , 'category_mid': , 'price': , 'detail': , 'region': , 'status': , 'file_count': , 'url':}
df = pd.DataFrame(data)
df

SyntaxError: invalid syntax (<ipython-input-3-eb070a956697>, line 3)